In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm
import random

## Part II: Estimation of Sensitivities in MC

In [2]:
def Delta_Analytical_Call(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

def Delta_Analytical_Digital(S, K, r, sigma, tau):
    d2 = (math.log(S/K) + (r-0.5*sigma**2)*tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d2) * math.e**(-r * tau)

In [27]:
@jit(nopython=True, fastmath=True, parallel=False)
def European_Call(T, K, r, S, sigma, trials, z):
    '''
    This function calculates the value of an European Call option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)

    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*z[i])
        
        if S_cur-K > 0:
            payoff_array[i] = (S_cur-K)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return np.mean(payoff_array)


@jit(nopython=True, fastmath=True, parallel=False)
def Digital_Call(T, K, r, S, sigma, trials, z):
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur > K:
            payoff_array[i] = 1 * math.e**(-r*T)
        else:
            payoff_array[i] = 0

    return np.mean(payoff_array)

In [4]:
def get_delta_call(kwargs, S, e, seed=None):
    z=np.zeros(kwargs['trials'])
    for i in range(kwargs['trials']):
        z[i] = np.random.normal()
        
    kwargs['z'] = z   
    kwargs['S'] = S
    
    V = European_Call(**kwargs)
    
    if not seed:
        for i in range(kwargs['trials']):
            z[i] = random.normalvariate(0, 1)
        kwargs['z'] = z   
        
    kwargs['S'] = S + e
    Ve = European_Call(**kwargs)
    
    return (Ve-V)/ e


def get_delta_digital(kwargs, S, e, seed=None):
    z=np.zeros(kwargs['trials'])
    for i in range(kwargs['trials']):
        z[i] = np.random.normal()
        
    kwargs['z'] = z   
    kwargs['S'] = S
    
    V = Digital_Call(**kwargs)
    
    if not seed:
        for i in range(kwargs['trials']):
            z[i] = random.normalvariate(0, 1)
        kwargs['z'] = z   
        
    kwargs['S'] = S + e
    
    Ve = Digital_Call(**kwargs)
    
    return (Ve-V)/ e

In [25]:
analytical = Delta_Analytical_Call(100, 99, 0.06, 0.2, 1)
analytical

0.6737355117348961

In [6]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**4

In [7]:
S = 100
e = .02

delta_matrix_noseeds = [get_delta_call(kwargs, S, e, None) for x in range(100)]
delta_matrix_seeds = [get_delta_call(kwargs, S, e, 100) for x in range(100)]

In [8]:
# Relative error with analytical value as reference
rel_error_same_seed = abs(analytical - np.mean(delta_matrix_seeds)) / analytical * 100
print(f"Relative error based on same seed: {rel_error_same_seed}%")

Relative error based on same seed: 0.024779201938914416%


In [9]:
rel_error_random_seed = abs(analytical - np.mean(delta_matrix_noseeds)) / analytical * 100
print(f"Relative error based on random seed: {rel_error_random_seed}%")

Relative error based on random seed: 160.6383256727763%


In [10]:
eps = [0.01,0.02,0.5]
size = [4,5,6,7]

In [11]:
seeded = np.matrix(np.zeros(12))
seeded.shape = (4,3)

unseeded = np.matrix(np.zeros(12))
unseeded.shape = (4,3)

In [20]:
def get_convergence_matrix(analytical):
    for row in range(len(size)):
        for column in range(len(eps)):
            kwargs['trials'] = 10**size[row]
            e = eps[column]
            seeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e, 100))) / analytical * 100
            unseeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e,None))) / analytical * 100
            
    return seeded, unseeded

#### Matrices as in lecture Monte-Carlo II (slides 24 and 26)

In [26]:
results = Parallel(n_jobs = 12)(delayed(get_convergence_matrix)(analytical) for i in range(12))

In [38]:
seeded[0].shape

(4, 3)

In [30]:
seeded = []
unseeded = []
for result in results:
    seeded.append(result[0])
    unseeded.append(result[1])

In [45]:
all_unseeded = np.matrix(np.zeros(12))
all_unseeded.shape = (4,3)

all_seeded = np.matrix(np.zeros(12))
all_seeded.shape = (4,3)

for mat in unseeded:
    for row in range(mat.shape[0]):
        for column in range(mat.shape[1]):
            all_unseeded[row, column] += mat[row, column]
            
for mat in seeded:
    for row in range(mat.shape[0]):
        for column in range(mat.shape[1]):
            all_seeded[row, column] += mat[row, column]

In [44]:
all_unseeded/12

matrix([[1887.14075769, 1271.04787763,   43.8699052 ],
        [ 808.99812597,  361.93327321,   16.87032737],
        [ 268.08016352,  150.54534793,    4.6293363 ],
        [  76.06796513,   39.58624691,    2.02105451]])

In [46]:
all_seeded/12

matrix([[0.51966368, 0.67735639, 0.82160354],
        [0.22908508, 0.21580141, 0.60682259],
        [0.06742423, 0.06758404, 0.6604558 ],
        [0.02357059, 0.03591098, 0.67053056]])

## Part 2.2

In [33]:
# Analytical digital
#https://448elmwood.files.wordpress.com/2015/04/study_pricing_digital_call_options1.pdf

analytical = Delta_Analytical_Digital(S=100, K=99, r=0.06, sigma=0.2, tau=1); analytical

0.5639320297620053

In [34]:
kwargs['trials'] = 10**4
S=100
e = 0.02
ans = [get_delta_digital(kwargs, S, e, 100) for x in range(100)]
np.mean(ans)

0.01944743761851597

In [35]:
digital = np.matrix(np.zeros(12))
digital.shape = (4,3)

eps = [0.01,0.02,0.5]
size = [4,5,6,7]

In [18]:
for row in range(len(size)):
    for column in range(len(eps)):
        kwargs['trials'] = 10**size[row]
        e = eps[column]
        digital[row,column] = abs(analytical - get_delta_digital(kwargs, S, e, 100)) / analytical * 100

In [22]:
digital

matrix([[ 87.0396115 ,  35.70513355,  94.42221159],
        [ 58.41708637,  89.47902185,  96.87710649],
        [ 96.02540826,  86.14737877,  97.00335822],
        [103.74580222,  97.54176511,  96.67176371]])